In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd 'drive/MyDrive/CS224N Project'

/content/drive/MyDrive/CS224N Project


In [4]:
import urllib
import bs4
import re
import json

In [5]:
NUM_PAGES = 10000

In [6]:
def get_movie_info(review_list):
    review_movie = review_list[1]
    movie_component = review_movie.find('strong')
    movie_name = movie_component.a.get_text()
    movie_url = movie_component.a.get('href')
    return movie_name, movie_url

def get_rating(review_list):
    review_movie = review_list[1]
    rating = review_movie.find('a', text='Overall Rating').next_sibling
    rating_value = float(rating.split()[1])
    return rating_value
    
def get_user_info(review_list):
    review_movie = review_list[1]
    user = review_movie.find('table').find('small').previous_sibling.previous_sibling
    user_name = user.text
    user_url = user.get('href')
    return user_name, user_url

def get_review_content(review_list):
    review_component = review_list[3]
    for div_elem in review_component.find_all('div'):
        div_elem.decompose()
    review_component.smooth()
    review_content = ' '.join(review_component.get_text().split())
    if review_content.endswith('read more'):
        review_content = review_content[:-9]
    review_content = review_content.strip()
    return review_content

def get_review(review_component):
    review_list = list(review_component.children)
    movie_name, movie_url = get_movie_info(review_list)
    rating_value = get_rating(review_list)
    user_name, user_url = get_user_info(review_list)
    review_content = get_review_content(review_list)
    return {
        'movie_name': movie_name,
        'movie_url': movie_url,
        'rating_value': rating_value,
        'user_name': user_name,
        'user_url': user_url,
        'review_content': review_content,
    }

In [7]:
def get_all_reviews(num_pages):
    reviews = []
    for i in range(num_pages):
        try:
            url = f'https://myanimelist.net/reviews.php?t=anime&p={i + 1}'
            source = urllib.request.urlopen(url).read()
            soup = bs4.BeautifulSoup(source, 'lxml')
            review_data = soup.find(id='content').find_all('div', class_='borderDark')
            for review_component in review_data:
                review = get_review(review_component)
                reviews.append(review)
        except:
            break
    return reviews

In [9]:
reviews = get_all_reviews(NUM_PAGES)
with open('reviews.txt', 'w') as f:
    json.dump(reviews, f)

[]


[]